In [12]:
import py_entitymatching as em
import pandas as pd
import numpy as np
import os

In [13]:
path_A = 'dataset/tracks_sample.csv'
path_B = 'datasets/songs_sample.csv'

In [14]:
# Read the CSV files
sample_A = em.read_csv_metadata(path_A,low_memory=False) # setting the parameter low_memory to False  to speed up loading.
sample_B = em.read_csv_metadata(path_B,low_memory=False)
# Set 'ID as the keys to the input tables
em.set_key(sample_A,'id')
em.set_key(sample_B,'id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [15]:
sample_A.head()

,id,movie_title,year,episode,song_title,artists
0,262144,the porter wagoner show,1961.0,the osborne brothers (#1.345),a picture from lifes other side,porter wagoner
1,2,#lawstinwoods,2013.0,the loop & rocks (#1.9)} {{suspended},unable to let go,bradley voorhees+mj rhythm
2,655363,the dark area,2000.0,NaN,tonight youll be mine,ernesto cortazar
3,655369,the dark chronicles,2011.0,NaN,in the pines,the stanley brothers
4,262154,the porter wagoner show,1961.0,the osborne brothers (#1.517),highway headed south,porter wagoner


In [16]:
sample_B.head()

,id,song_title,artists,year
0,552934,science fiction,divinyls,1983
1,652425,the sound,new monsoon,2005
2,570585,the devil,geri x.,0
3,619428,xiang dui wu yan,elva hsiao,0
4,543601,we will rock you,andrew spencer,0


In [17]:
block_f = em.get_features_for_blocking(sample_A, sample_B)

In [18]:
def title_function(x, y):

    x_title = str(x['song_title'])
    y_title = str(y['song_title'])
    
    if (x_title in y_title) or (y_title in x_title):
        return False
    else:
        x_split = x_title.split()
        y_split = y_title.split()
        
        intersection = len(set(x_split) & set(y_split))
        union = len(set(x_split) | set(y_split))
        
        #if(union - intersection > 3):
        #    return True
        #else:
        if(intersection / union < 0.25):
            return True
        else:
            return False

In [19]:
def artists_function(x, y):

    x_artists = str(x['artists'])
    y_artists = str(y['artists'])
    
    if (x_artists in y_artists) or (y_artists in x_artists):
        return False
    else:
        x_split = x_artists.split()
        y_split = y_artists.split()
        
        intersection = len(set(x_split) & set(y_split))
        union = len(set(x_split) | set(y_split))
        
        #if(union - intersection > 4):
        #    return True
        #else:
        if(intersection / union < 0.25):
            return True
        else:
            return False

In [20]:
rb = em.RuleBasedBlocker()
ob = em.OverlapBlocker()
bb = em.BlackBoxBlocker()

# remove pairs that don't share similar titles
rule1 = ['song_title_song_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1']
#rule2 = ['artists_artists_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.2']
rb.add_rule(rule1, block_f)
#rb.add_rule(rule2, block_f)

C1 = rb.block_tables(sample_A, sample_B, l_output_attrs=['song_title','year','artists'], r_output_attrs=['song_title','year','artists'], show_progress=False)


0%                          100%
[##############################] | ETA: 01:36:20 | ETA: 01:31:33 | ETA: 01:28:47 | ETA: 01:25:00 | ETA: 01:20:57 | ETA: 01:17:09 | ETA: 01:13:15 | ETA: 01:09:28 | ETA: 01:06:01 | ETA: 01:02:27 | ETA: 00:59:20 | ETA: 00:55:58 | ETA: 00:52:39 | ETA: 00:49:55 | ETA: 00:47:29 | ETA: 00:45:33 | ETA: 00:42:45 | ETA: 00:39:25 | ETA: 00:35:57 | ETA: 00:32:32 | ETA: 00:29:29 | ETA: 00:26:06 | ETA: 00:22:46 | ETA: 00:19:28 | ETA: 00:16:09 | ETA: 00:13:19 | ETA: 00:09:57 | ETA: 00:06:36 | ETA: 00:03:17 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 01:38:22
0%  100%
[  ]

Finding pairs with missing value...


[##] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


In [21]:
bb.set_black_box_function(title_function)
C2 = bb.block_candset(C1)

bb.set_black_box_function(artists_function)
C3 = bb.block_candset(C2)

0%                          100%
[##############################] | ETA: 00:45:26 | ETA: 00:42:59 | ETA: 00:39:35 | ETA: 00:36:33 | ETA: 00:33:30 | ETA: 00:31:08 | ETA: 00:29:06 | ETA: 00:27:19 | ETA: 00:25:42 | ETA: 00:24:12 | ETA: 00:22:47 | ETA: 00:21:24 | ETA: 00:20:05 | ETA: 00:18:47 | ETA: 00:17:31 | ETA: 00:16:16 | ETA: 00:15:03 | ETA: 00:13:50 | ETA: 00:12:38 | ETA: 00:11:27 | ETA: 00:10:16 | ETA: 00:09:06 | ETA: 00:07:57 | ETA: 00:06:48 | ETA: 00:05:39 | ETA: 00:04:30 | ETA: 00:03:22 | ETA: 00:02:14 | ETA: 00:01:07 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:33:38
0%                          100%
[##############################] | ETA: 00:07:34 | ETA: 00:06:06 | ETA: 00:05:21 | ETA: 00:04:53 | ETA: 00:04:32 | ETA: 00:04:15 | ETA: 00:03:59 | ETA: 00:03:45 | ETA: 00:03:32 | ETA: 00:03:20 | ETA: 00:03:09 | ETA: 00:02:57 | ETA: 00:02:47 | ETA: 00:02:36 | ETA: 00:02:25 | ETA: 00:02:15 | ETA: 00:02:05 | ETA: 00:01:55 | ETA: 00:01:45 | ETA: 00:01:35 | ETA: 00:01:25 | ETA: 

In [22]:
dbg = em.debug_blocker(C2, sample_A, sample_B, output_size=50)

In [23]:
C3.to_csv('pairs_passed_stage4_v2.csv', index = False, sep = ',')

In [24]:
G = em.label_table(C3, label_column_name='gold_labels')
G.to_csv('labeled_data_stage4_v2.csv', index = False, sep = ',')

Column name (gold_labels) is not present in dataframe


In [ ]:
G = em.label_table(C2, label_column_name='gold_labels')